# Part 4 - Nobel prize winners who were born in the same city as their spouses 


You may find the predicate `'<isMarriedTo>'` useful to create a Dataframe of all mariages.
Please also show the cities in which the Nobel laureates and their spouses were born.

Please sort the output alphabetically by the person (prize winner) column.

In [1]:
from pyspark.sql import SQLContext
from pyspark.sql.types import *
sqlContext = SQLContext(sc)

In [2]:
# After having looked at the head of the file in the terminal, one can see that there is no header for the columns
# so we'll have to create the schema ourselves. Also, the first line is an explanation of what the database contains,
# which has to be removed before converting it into a df, so we'll upload the data to an RDD to do that removal
# with the following function:
def remove_first(itr_index, itr):
    return iter(list(itr)[1:]) if itr_index == 0 else itr

In [3]:
# Import Data to an RDD and remove first line
file = sc.textFile("/yago/yagoFacts.tsv")
data = file.mapPartitionsWithIndex(remove_first) # Maybe this is smarter: rdd.zipWithIndex().filter(lambda tup: tup[1] > 0).map(lambda tup: tup[0])

In [4]:
# Split by columns
rdd_map = data.map(lambda x: x.split("\t"))   
#rdd_map.take(3)

In [5]:
# We've already seen the data and datatypes it contains, because there's no header describing the dataset
# we create a schema
schema = StructType([
                StructField("id",StringType()),
                StructField("subject", StringType()),
                StructField("predicate", StringType()),
                StructField("object", StringType()),
                StructField("value", StringType())#DoubleType
                    ])
# It didn't work with the value as a DoubleType, so we'll create it with that column as a string
# and change the datatype afterwards
df = sqlContext.createDataFrame(rdd_map, schema)
#df.take(3)

In [6]:
# Change value Datatype
df = df.withColumn("value", df["value"].cast(DoubleType()))
#df.take(3)

Now for the Transitive Type Dataset

In [7]:
file = sc.textFile("/yago/yagoTransitiveType.tsv")
data2 = file.mapPartitionsWithIndex(remove_first) # Maybe this is smarter:
#data = file.zipWithIndex().filter(lambda tup: tup[1] > 0).map(lambda tup: tup[0])
rdd_map2 = data2.map(lambda x: x.split("\t"))   
schema = StructType([
                StructField("id",StringType()),
                StructField("subject", StringType()),
                StructField("predicate", StringType()),
                StructField("object", StringType()),
                StructField("value", StringType())#DoubleType
                    ])
# It didn't work with the value as a DoubleType, so we'll create it with that column as a string
# and change the datatype afterwards
df_subclasses = sqlContext.createDataFrame(rdd_map2, schema)
df_subclasses = df_subclasses.withColumn("value", df_subclasses["value"].cast(DoubleType()))
#df_subclasses.show(3)

In [8]:
# Import necessary libraries
import graphframes
from graphframes import *

import matplotlib.pyplot as plt
%matplotlib inline

from pyspark.sql.functions import col, lit, when
from pyspark.sql import Row

from datetime import datetime
import re
import numpy as np

## Create Datasets

In [9]:
df.createOrReplaceTempView("df")
df_subclasses.createOrReplaceTempView("df_subclasses")

### Nobel Laureates

In [10]:
laureates = df.select("subject", "object", "predicate").filter("predicate = '<hasWonPrize>' AND object = '<Nobel_Prize_in_Chemistry>' OR object = '<Nobel_Prize_in_Physics>' OR object = '<Nobel_Prize_in_Literature>' OR object = '<Nobel_Prize>' OR object = '<Nobel_Memorial_Prize_in_Economic_Sciences>' OR object = '<Nobel_Prize_in_Physiology_or_Medicine>'")

In [11]:
laureates.createOrReplaceTempView("laureates")

### Place of birth

In [12]:
born = df.select("subject", "object", "predicate").filter("predicate = '<wasBornIn>'")

In [13]:
born.createOrReplaceTempView("born")

### Nobels with Place of Birth

In [14]:
laureates_born = spark.sql("SELECT l.subject AS laureate, l.object AS nobel, l.predicate AS nobel_pred, \
b.object AS city, b.predicate AS city_pred \
FROM laureates l JOIN born b \
ON l.subject = b.subject")

In [15]:
laureates_born.columns

['laureate', 'nobel', 'nobel_pred', 'city', 'city_pred']

In [16]:
laureates_born.createOrReplaceTempView("laureates_born")

### All Nobels, Place of Birth, and Person married to

In [17]:
laureates_born_married = spark.sql("SELECT l.laureate, l.nobel, l.nobel_pred, l.city, l.city_pred, \
d.object AS pers_married, d.predicate AS married_to_pred \
FROM laureates_born l JOIN df d ON l.laureate = d.subject \
WHERE d.predicate = '<isMarriedTo>'")

In [18]:
laureates_born_married.columns

['laureate',
 'nobel',
 'nobel_pred',
 'city',
 'city_pred',
 'pers_married',
 'married_to_pred']

In [19]:
laureates_born_married.createOrReplaceTempView("laureates_born_married")

### Filter to have married couples born is same city

In [20]:
laureates_born_married_both = spark.sql("SELECT l.laureate, l.nobel, l.nobel_pred, l.city, l.city_pred, l.pers_married, \
l.married_to_pred \
FROM laureates_born_married l JOIN df d ON l.pers_married = d.subject \
WHERE d.predicate = '<wasBornIn>' AND l.city = d.object")

In [21]:
#laureates_born_married_both.show(10)

### Create vertex, edges and graph

In [22]:
v = laureates_born_married_both.select("laureate").withColumnRenamed("laureate", "id")\
.union(laureates_born_married_both.select("nobel").withColumnRenamed("nobel", "id"))\
.union(laureates_born_married_both.select("city").withColumnRenamed("city", "id"))\
.union(laureates_born_married_both.select("pers_married").withColumnRenamed("pers_married", "id")).distinct()

In [23]:
e = laureates_born_married_both.select("laureate", "nobel", "nobel_pred").withColumnRenamed("laureate", "src")\
.withColumnRenamed("nobel", "dst").withColumnRenamed("nobel_pred", "pred")\
.union(laureates_born_married_both.select("laureate", "city", "city_pred").withColumnRenamed("laureate", "src")\
.withColumnRenamed("city", "dst").withColumnRenamed("city_pred", "pred"))\
.union(laureates_born_married_both.select("laureate", "pers_married", "married_to_pred").withColumnRenamed("laureate", "src")\
.withColumnRenamed("pers_married", "dst").withColumnRenamed("married_to_pred", "pred"))\
.union(laureates_born_married_both.select("pers_married", "city", "city_pred").withColumnRenamed("pers_married", "src")\
.withColumnRenamed("city", "dst").withColumnRenamed("city_pred", "pred")).distinct()

In [24]:
g = GraphFrame(v, e)

In [ ]:
g.find("(l)-[e1]->(n); (l)-[e2]->(p); (l)-[e3]->(b); (m)-[e4]->(b)")\
.filter("e1.pred = '<hasWonPrize>'").filter("e2.pred = '<isMarriedTo>'")\
.filter("e3.pred = '<wasBornIn>'").filter("e4.pred = '<wasBornIn>'")\
.filter("l.id != m.id").sort("l").show(20)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+--------------------+
|                   l|                  e1|                   n|                  e2|                   p|                  e3|                b|                   m|                  e4|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+--------------------+
|[<Carl_Ferdinand_...|[<Carl_Ferdinand_...|[<Nobel_Prize_in_...|[<Carl_Ferdinand_...|      [<Gerty_Cori>]|[<Carl_Ferdinand_...|       [<Prague>]|      [<Gerty_Cori>]|[<Gerty_Cori>, <P...|
|[<Frédéric_Joliot...|[<Frédéric_Joliot...|[<Nobel_Prize_in_...|[<Frédéric_Joliot...|[<Irène_Joliot-Cu...|[<Frédéric_Joliot...|        [<Paris>]|[<Irène_Joliot-Cu...|[<Irène_Joliot-Cu...|
|      [<Gerty_Cori>]|[<Gerty_Cori>, <N...|[<Nobel_Prize_in_